# Create a dictionary and use Txy diagrams

## 1. Introduction

This Jupyter Notebook creates a property package for the generic property package and utilized the Txy diagrams for the generic properties. A dictionary with the neccesary parameters.

### 1.1 Tutorial objectives

The goals of this tutorial are:

* Create a parameter block directly on a jupyter notebook.
* Utilize the Generic Property Package, which provides a flexible platform for users to build property packages by calling upon libraries of modular sub-models to build up complex property calculations with the least effort possible.
* Demonstrate the use of Txy diagrams in the IDAES utilities.

## 2. Problem Statement

### 2.1 Setting up the problem in IDAES

In the next cell, we will be importing the necessary components from Pyomo and IDAES.

In [1]:
# Import objects from pyomo package
from pyomo.environ import ConcreteModel

# Import idaes logger to set output levels
import idaes.logger as idaeslog

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import GenericParameterBlock

### 2.2 Importing property sub-libraries

The IDAES generic property package calls upon libraries of modular sub-models to build up complex property calculations with the least effort possible. We have to import these libraries.

In [2]:
# Import Python libraries
import logging

# Import Pyomo units
from pyomo.environ import units as pyunits
from pyomo.environ import SolverFactory

from idaes.core import FlowsheetBlock

# Import IDAES cores
from idaes.core import LiquidPhase, VaporPhase, Component

from idaes.generic_models.properties.core.state_definitions import FTPx
from idaes.generic_models.properties.core.eos.ideal import Ideal
from idaes.generic_models.properties.core.phase_equil import smooth_VLE
from idaes.generic_models.properties.core.phase_equil.bubble_dew import \
        IdealBubbleDew,LogBubbleDew
from idaes.generic_models.properties.core.eos.ceos import Cubic, CubicType, MixingRuleB
from idaes.generic_models.properties.core.phase_equil.forms import fugacity,log_fugacity

import idaes.generic_models.properties.core.pure.Perrys as Perrys
import idaes.generic_models.properties.core.pure.RPP4 as RPP4

# Import unit models from the model library
from idaes.generic_models.unit_models import Flash

### 2.3 Creating a generic property parameter block

The next step is to give all the parameter information required for the sub-libraries. Here we will specify the base units of our model, the bounds on the state properties and the reference conditions of our properties.

In [3]:
# Add properties parameter blocks to the flowsheet with specifications -- pull from .py file
# from idaes.generic_models.properties.core.examples.BT_VDW import configuration

In [4]:
#Make configuration from original Dictionary_Txy_diagrams_AG
configuration = {
    # Specifying components
    "components": {
        'benzene': {"type": Component,
                    "elemental_composition": {"C": 6, "H": 6},
                    "dens_mol_liq_comp": Perrys,
                    "enth_mol_liq_comp": Perrys,
                    "enth_mol_ig_comp": RPP4,
                    "pressure_sat_comp": RPP4,
                    "phase_equilibrium_form": {("Vap", "Liq"): fugacity},
                    "parameter_data": {
                        "mw": (78.1136E-3, pyunits.kg/pyunits.mol),  # [1]
                        "pressure_crit": (48.9e5, pyunits.Pa),  # [1]
                        "temperature_crit": (562.2, pyunits.K),  # [1]
                        "omega": 0.212,  # [1]
                        "dens_mol_liq_comp_coeff": {
                            '1': (1.0162, pyunits.kmol*pyunits.m**-3),  # [2] pg. 2-98
                            '2': (0.2655, None),
                            '3': (562.16, pyunits.K),
                            '4': (0.28212, None)},
                        "cp_mol_ig_comp_coeff": {
                            'A': (-3.392E1, pyunits.J/pyunits.mol/pyunits.K),  # [1]
                            'B': (4.739E-1, pyunits.J/pyunits.mol/pyunits.K**2),
                            'C': (-3.017E-4, pyunits.J/pyunits.mol/pyunits.K**3),
                            'D': (7.130E-8, pyunits.J/pyunits.mol/pyunits.K**4)},
                        "cp_mol_liq_comp_coeff": {
                            '1': (1.29E2, pyunits.J/pyunits.kmol/pyunits.K),  # [2]
                            '2': (-1.7E-1, pyunits.J/pyunits.kmol/pyunits.K**2),
                            '3': (6.48E-4, pyunits.J/pyunits.kmol/pyunits.K**3),
                            '4': (0, pyunits.J/pyunits.kmol/pyunits.K**4),
                            '5': (0, pyunits.J/pyunits.kmol/pyunits.K**5)},
                        "enth_mol_form_liq_comp_ref": (
                            49.0e3, pyunits.J/pyunits.mol),  # [3]
                        "enth_mol_form_vap_comp_ref": (
                            82.9e3, pyunits.J/pyunits.mol),  # [3]
                        "pressure_sat_comp_coeff": {'A': (-6.98273, None),  # [1]
                                                    'B': (1.33213, None),
                                                    'C': (-2.62863, None),
                                                    'D': (-3.33399, None)}}},
        'toluene': {"type": Component,
                    "elemental_composition": {"C": 7, "H": 8},
                    "dens_mol_liq_comp": Perrys,
                    "enth_mol_liq_comp": Perrys,
                    "enth_mol_ig_comp": RPP4,
                    "pressure_sat_comp": RPP4,
                    "phase_equilibrium_form": {("Vap", "Liq"): fugacity},
                    "parameter_data": {
                        "mw": (92.1405E-3, pyunits.kg/pyunits.mol),  # [1]
                        "pressure_crit": (41e5, pyunits.Pa),  # [1]
                        "temperature_crit": (591.8, pyunits.K),  # [1]
                        "omega": 0.263,  # [1]
                        "dens_mol_liq_comp_coeff": {
                            '1': (0.8488, pyunits.kmol*pyunits.m**-3),  # [2] pg. 2-98
                            '2': (0.26655, None),
                            '3': (591.8, pyunits.K),
                            '4': (0.2878, None)},
                        "cp_mol_ig_comp_coeff": {
                            'A': (-2.435E1, pyunits.J/pyunits.mol/pyunits.K),  # [1]
                            'B': (5.125E-1, pyunits.J/pyunits.mol/pyunits.K**2),
                            'C': (-2.765E-4, pyunits.J/pyunits.mol/pyunits.K**3),
                            'D': (4.911E-8, pyunits.J/pyunits.mol/pyunits.K**4)},
                        "cp_mol_liq_comp_coeff": {
                            '1': (1.40E2, pyunits.J/pyunits.kmol/pyunits.K),  # [2]
                            '2': (-1.52E-1, pyunits.J/pyunits.kmol/pyunits.K**2),
                            '3': (6.95E-4, pyunits.J/pyunits.kmol/pyunits.K**3),
                            '4': (0, pyunits.J/pyunits.kmol/pyunits.K**4),
                            '5': (0, pyunits.J/pyunits.kmol/pyunits.K**5)},
                        "enth_mol_form_liq_comp_ref": (
                            12.0e3, pyunits.J/pyunits.mol),  # [3]
                        "enth_mol_form_vap_comp_ref": (
                            50.1e3, pyunits.J/pyunits.mol),  # [3]
                        "pressure_sat_comp_coeff": {'A': (-7.28607, None),  # [1]
                                                    'B': (1.38091, None),
                                                    'C': (-2.83433, None),
                                                    'D': (-2.79168, None)}}}},

    # Specifying phases
    "phases":  {'Liq': {"type": LiquidPhase,
                        "equation_of_state": Cubic,
                        "equation_of_state_options": {
                            "type": CubicType.PR,
                            "mixing_rule_b": MixingRuleB.default}},
                'Vap': {"type": VaporPhase,
                        "equation_of_state": Cubic,
                        "equation_of_state_options": {
                            "type": CubicType.PR,
                            "mixing_rule_b": MixingRuleB.default}}},

    # Set base units of measurement
    "base_units": {"time": pyunits.s,
                   "length": pyunits.m,
                   "mass": pyunits.kg,
                   "amount": pyunits.mol,
                   "temperature": pyunits.K},

    # Specifying state definition
    "state_definition": FTPx,
    "state_bounds": {"flow_mol": (0, 100, 1000, pyunits.mol/pyunits.s),
                     "temperature": (273.15, 300, 450, pyunits.K),
                     "pressure": (5e4, 1e5, 1e6, pyunits.Pa)},
    "pressure_ref": (1e5, pyunits.Pa),
    "temperature_ref": (300, pyunits.K),

    # Defining phase equilibria
    "phases_in_equilibrium": [("Vap", "Liq")],
    "phase_equilibrium_state": {("Vap", "Liq"): smooth_VLE},
    "bubble_dew_method": LogBubbleDew,
    "parameter_data": {"PR_kappa": {("benzene", "benzene"): 0.000,
                                    ("benzene", "toluene"): 0.0058,
                                    ("toluene", "benzene"): 0.011,
                                    ("toluene", "toluene"): 0.000}}}

### 2.4 Building the model

In the next cell, we will first create a model and attach a the property package. 

In [5]:
# Create the ConcreteModel
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})

2022-02-18 15:51:36 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.
2022-02-18 15:51:36 [WARNING] idaes.generic_models.properties.core.pure.Perrys: DEPRECATED - fs.properties.benzene dens_mol_liq_comp_coeff index 'eqn_type' should be specified, defaulting to equation form 1.
2022-02-18 15:51:36 [WARNING] idaes.generic_models.properties.core.pure.Perrys: DEPRECATED - fs.properties.toluene dens_mol_liq_comp_coeff index 'eqn_type' should be specified, defaulting to equation form 1.


In [6]:
m.fs.state_block.flow_mol.fix(10)
m.fs.state_block.temperature.fix(300)
m.fs.state_block.mole_frac_comp["benzene"].fix(0.8) 
m.fs.state_block.mole_frac_comp["toluene"].fix(0.2) 
m.fs.state_block.pressure.fix(100000)

In [7]:
m.fs.state_block.initialize()
solver = SolverFactory('ipopt')
result = solver.solve(m, tee=True)
m.display()

2022-02-18 15:51:36 [INFO] idaes.init.fs.state_block: Starting initialization
2022-02-18 15:51:37 [INFO] idaes.init.fs.state_block: Dew and bubble point initialization: optimal - Optimal Solution Found.
2022-02-18 15:51:37 [INFO] idaes.init.fs.state_block: Equilibrium temperature initialization completed.
2022-02-18 15:51:38 [INFO] idaes.init.fs.state_block: Phase equilibrium initialization: optimal - Optimal Solution Found.
2022-02-18 15:51:39 [INFO] idaes.init.fs.state_block: Property initialization: optimal - Optimal Solution Found.
2022-02-18 15:51:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version 

                Key                       : Lower : Body                   : Upper
                ('Vap', 'Liq', 'benzene') :   0.0 : 3.6193270602780103e-14 :   0.0
                ('Vap', 'Liq', 'toluene') :   0.0 : -7.038813976123492e-14 :   0.0
            log_mole_frac_tdew_eqn : Size=2
                Key                       : Lower : Body                   : Upper
                ('Vap', 'Liq', 'benzene') :   0.0 : 1.1102230246251565e-16 :   0.0
                ('Vap', 'Liq', 'toluene') :   0.0 :                    0.0 :   0.0
            eq_mole_frac_tdew : Size=1
                Key            : Lower : Body : Upper
                ('Vap', 'Liq') :   1.0 :  1.0 :   1.0
            equilibrium_constraint : Size=2
                Key                       : Lower : Body                   : Upper
                ('Vap', 'Liq', 'benzene') :   0.0 :    7.8580342233181e-10 :   0.0
                ('Vap', 'Liq', 'toluene') :   0.0 : -5.475158104673028e-10 :   0.0
